# Tips

- Split the code into multiple cells (multiple cells can be executed independently and easily debugged)

- Try to use `os.path.join` to join paths instead of template string (The file path may be wrong when you run the code in other systems)

- Load the template out of the loop to improve efficiency (loaded once and used repeatedly). (In this code, the template is loaded repeatedly.)

In [2]:
import pandas as pd
from docxtpl import DocxTemplate
import os
import subprocess



PERSONAL_NAME = "Dehao Guo"  
TEMPLATE_PATH = "template.docx"  # Word模板路径
DATA_DIR = "application_data"  # 存储Excel数据的文件夹
OUTPUT_ROOT = os.path.expanduser("~/HW_School_Application")  # 主输出目录
LIBREOFFICE_PATH = "/Applications/LibreOffice.app/Contents/MacOS/soffice"  # LibreOffice路径（macOS默认）


# 步骤1-6：生成Excel数据文件
# --------------------------
def generate_excel_data():
    # 创建数据文件夹
    os.makedirs(DATA_DIR, exist_ok=True)
    
    # 步骤3：30所大学（10所Top30，10所Top60，10所Top90）
    universities = [
        # Top30
        "Massachusetts Institute of Technology", "Stanford University", "University of Chicago",
        "Princeton University", "University of California-Berkeley", "Yale University",
        "Northwestern University", "Columbia University", "University of Pennsylvania", "Harvard University",
        # Top60
        "New York University", "University of California-Los Angeles", "University of Michigan",
        "University of Wisconsin-Madison", "University of Minnesota", "University of Toronto",
        "London School of Economics", "University of California-San Diego", "University of Virginia", "Cornell University",
        # Top90
        "University of North Carolina", "University of Texas-Austin", "Brown University",
        "Duke University", "University of Washington", "Boston University",
        "University of Illinois", "Ohio State University", "Pennsylvania State University", "Rice University"
    ]
    pd.DataFrame({"university": universities}).to_excel(
        f"{DATA_DIR}/universities.xlsx", index=False
    )
    
    # 步骤4：3个研究领域
    research_areas = ["Financial Economics", "Data Science in Management", "Behavioral Finance"]
    pd.DataFrame({"research_area": research_areas}).to_excel(
        f"{DATA_DIR}/research_areas.xlsx", index=False
    )
    
    # 步骤5：每个领域的3本顶级期刊
    journals_df = pd.DataFrame({
        "research_area": research_areas,
        "journal1": [
            "Journal of Finance", 
            "Management Science", 
            "Journal of Financial and Quantitative Analysis"
        ],
        "journal2": [
            "Review of Financial Studies", 
            "Information Systems Research", 
            "Journal of Behavioral Finance"
        ],
        "journal3": [
            "Journal of Financial Economics", 
            "MIS Quarterly", 
            "Experimental Economics"
        ]
    })
    journals_df.to_excel(f"{DATA_DIR}/journals.xlsx", index=False)
    
    # 步骤6：每个领域的技能、目标专业和职业目标
    skills_df = pd.DataFrame({
        "research_area": research_areas,
        "skills": [
            "Python, R, Econometrics, Time Series Analysis, SQL",
            "Python, Machine Learning, Big Data Analytics, Tableau, Database Management",
            "Behavioral Analysis, Statistical Modeling, Python, Financial Modeling, Psychometrics"
        ],
        "program": [
            "Master of Financial Economics",
            "Master of Science in Management (Data Science Track)",
            "Master of Behavioral Finance"
        ],
        "career_goal": [
            "financial economist",
            "data science manager",
            "behavioral finance researcher"
        ]
    })
    skills_df.to_excel(f"{DATA_DIR}/skills.xlsx", index=False)
    
    print("Excel数据文件生成完成")


# --------------------------
# 步骤7-8：批量生成Word文档
# --------------------------
def generate_word_documents():
    # 创建输出目录
    word_output = os.path.join(OUTPUT_ROOT, "word_files")
    os.makedirs(word_output, exist_ok=True)
    
    # 读取数据
    universities = pd.read_excel(f"{DATA_DIR}/universities.xlsx")["university"].tolist()
    research_areas = pd.read_excel(f"{DATA_DIR}/research_areas.xlsx")["research_area"].tolist()
    journals_df = pd.read_excel(f"{DATA_DIR}/journals.xlsx")
    skills_df = pd.read_excel(f"{DATA_DIR}/skills.xlsx")
    
    # 合并研究领域数据（期刊+技能）
    area_info = {}
    for area in research_areas:
        journals = journals_df[journals_df["research_area"] == area].iloc[0]
        skills = skills_df[skills_df["research_area"] == area].iloc[0]
        area_info[area] = {
            "journal1": journals["journal1"],
            "journal2": journals["journal2"],
            "journal3": journals["journal3"],
            "skills": skills["skills"],
            "program": skills["program"],
            "career_goal": skills["career_goal"]
        }
    
    # 循环生成Word
    count = 0
    for uni in universities:
        for area in research_areas:
            count += 1
            # 填充模板的数据
            context = {
                "name": "Dehao Guo",
                "university": uni,
                "research_area": area,
                **area_info[area]
            }
            
            # 生成并保存Word
            doc = DocxTemplate(TEMPLATE_PATH)
            doc.render(context)
            # 文件名处理（去除空格和特殊字符）
            safe_uni = uni.replace(" ", "_").replace(",", "").replace("-", "_")
            safe_area = area.replace(" ", "_").replace(",", "").replace("-", "_")
            word_path = os.path.join(word_output, f"SoP_{safe_uni}_{safe_area}.docx")
            doc.save(word_path)
            
            if count % 10 == 0:  # 每生成10个提示一次
                print(f"已生成 {count}/90 份Word文档")
    
    print(f"✅ 全部Word文档生成完成，共 {count} 份")
    return word_output  # 返回Word文件目录


# --------------------------
# 步骤9：用LibreOffice转换为PDF（基于macOS）
# --------------------------
def convert_to_pdf(word_dir):
    # 创建PDF输出目录
    pdf_output = os.path.join(OUTPUT_ROOT, "pdf_files")
    os.makedirs(pdf_output, exist_ok=True)
    
    # 检查LibreOffice是否存在
    if not os.path.exists(LIBREOFFICE_PATH):
        print("未找到LibreOffice，请检查路径是否正确")
        return
    
    # 批量转换
    count = 0
    for filename in os.listdir(word_dir):
        if filename.endswith(".docx"):
            count += 1
            word_path = os.path.join(word_dir, filename)
            # 调用LibreOffice命令行转换
            try:
                subprocess.run(
                    [
                        LIBREOFFICE_PATH,
                        "--headless",  # 无界面模式
                        "--convert-to", "pdf",  # 转换格式
                        "--outdir", pdf_output,  # 输出目录
                        word_path  # 输入文件
                    ],
                    check=True,
                    stdout=subprocess.PIPE,  # 隐藏输出信息
                    stderr=subprocess.PIPE
                )
                if count % 10 == 0:
                    print(f"已转换 {count}/90 份PDF文档")
            except Exception as e:
                print(f"转换失败 {filename}：{str(e)}")
    
    print(f"✅ PDF转换完成，共 {count} 份")


# --------------------------
# 主函数：按步骤执行
# --------------------------
if __name__ == "__main__":
    # 步骤1-6：生成Excel数据
    generate_excel_data()
    
    # 步骤7-8：生成Word文档
    word_directory = generate_word_documents()
    
    # 步骤9：转换为PDF（macOS通过LibreOffice）
    convert_to_pdf(word_directory)
    
    print(f"\n所有操作完成，文件保存于：{OUTPUT_ROOT}")

Excel数据文件生成完成
已生成 10/90 份Word文档
已生成 20/90 份Word文档
已生成 30/90 份Word文档
已生成 40/90 份Word文档
已生成 50/90 份Word文档
已生成 60/90 份Word文档
已生成 70/90 份Word文档
已生成 80/90 份Word文档
已生成 90/90 份Word文档
✅ 全部Word文档生成完成，共 90 份
已转换 10/90 份PDF文档
已转换 20/90 份PDF文档
已转换 30/90 份PDF文档
已转换 40/90 份PDF文档
已转换 50/90 份PDF文档
已转换 60/90 份PDF文档
已转换 70/90 份PDF文档
已转换 80/90 份PDF文档
已转换 90/90 份PDF文档
✅ PDF转换完成，共 90 份

所有操作完成，文件保存于：/Users/guodehao/HW_School_Application
